In [2]:
%matplotlib inline
import pandas
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from torch.utils.data import Dataset
from IPython.display import display
from collections import defaultdict
from sklearn.metrics import accuracy_score

In [53]:
data_df = pd.read_csv('clean-phone-data-for-students.csv')

In [54]:
display(data_df.head())
data_df.describe()

,Sentence Utterance,Action,Object
0,<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counte...,enquire,payment
1,internet ยังความเร็วอยุ่เท่าไหร ครับ,enquire,package
2,ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้...,report,suspend
3,พี่ค่ะยังใช้ internet ไม่ได้เลยค่ะ เป็นเครื่อ...,enquire,internet
4,ฮาโหล คะ พอดีว่าเมื่อวานเปิดซิมทรูมูฟ แต่มันโ...,report,phone_issues


,Sentence Utterance,Action,Object
count,16175,16175,16175
unique,13389,10,33
top,บริการอื่นๆ,enquire,service
freq,97,10377,2525


In [55]:
data_df = data_df[["Sentence Utterance", "Object"]]
data_df.columns = ['input', 'raw_label']
display(data_df.describe())
display(data_df.raw_label.unique())

,input,raw_label
count,16175,16175
unique,13389,33
top,บริการอื่นๆ,service
freq,97,2525


array(['payment', 'package', 'suspend', 'internet', 'phone_issues',
       'service', 'nonTrueMove', 'balance', 'detail', 'bill', 'credit',
       'promotion', 'mobile_setting', 'iservice', 'roaming', 'truemoney',
       'information', 'lost_stolen', 'balance_minutes', 'idd',
       'TrueMoney', 'garbage', 'Payment', 'IDD', 'ringtone', 'Idd',
       'rate', 'loyalty_card', 'contact', 'officer', 'Balance', 'Service',
       'Loyalty_card'], dtype=object)

In [56]:
data_df['clean_label']=data_df['raw_label'].str.lower().copy()
data_df.drop('raw_label', axis=1, inplace=True)
display(data_df.describe())
display(data_df.clean_label.unique())

,input,clean_label
count,16175,16175
unique,13389,26
top,บริการอื่นๆ,service
freq,97,2528


array(['payment', 'package', 'suspend', 'internet', 'phone_issues',
       'service', 'nontruemove', 'balance', 'detail', 'bill', 'credit',
       'promotion', 'mobile_setting', 'iservice', 'roaming', 'truemoney',
       'information', 'lost_stolen', 'balance_minutes', 'idd', 'garbage',
       'ringtone', 'rate', 'loyalty_card', 'contact', 'officer'],
      dtype=object)

In [57]:
data_df = data_df.drop_duplicates("input", keep="first")
data_df = data_df.reset_index(drop=True)
display(data_df.describe())

,input,clean_label
count,13389,13389
unique,13389,26
top,<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counte...,service
freq,1,2111


# Model 1 TF-IDF

Build a model to train a tf-idf text classifier. Use a simple logistic regression model for the classifier.

For this part, you may find this [tutorial](https://scikit-learn.org/stable/auto_examples/text/plot_document_classification_20newsgroups.html#sphx-glr-auto-examples-text-plot-document-classification-20newsgroups-py) helpful.

Below are some design choices you need to consider to accomplish this task. Be sure to answer them when you submit your model.

What tokenizer will you use? Why?

**Ans:**

Will you ignore some stop words (a, an, the, to, etc. for English) in your tf-idf? Is it important?
PythaiNLP provides a list of stopwords if you want to use (https://pythainlp.org/docs/2.0/api/corpus.html#pythainlp.corpus.common.thai_stopwords)

**Ans:**

The dictionary of TF-IDF is usually based on the training data. How many words in the test set are OOVs?

**Ans:**

In [64]:
import deepcut
import attacut
import time
from tqdm import tqdm
import pythainlp

In [37]:
# 19 secs for 100 samples
# 13389 samples = 42 mins; ain't nobody got time for that
start_time = time.time()
for i in range(100):
    # print(f"original: {data_df['input'][i]}")
    # print(deepcut.tokenize(data_df['input'][i]))
    deepcut.tokenize(data_df['input'][i])
end_time = time.time()

print(f"Total time: {end_time - start_time}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
3/3 ━━━━━━

In [ ]:
# 2.25 secs for 100 samples
# 13389 samples = 5 mins; now we're talking
start_time = time.time()
for i in range(100):
    attacut.tokenize(data_df['input'][0])
end_time = time.time()

print(f"Total time: {end_time - start_time}")


Total time: 2.259876012802124


In [69]:
data = data_df.copy().to_numpy()

unique_label = data_df.clean_label.unique()

label_2_num_map = dict(zip(unique_label, range(len(unique_label))))
num_2_label_map = dict(zip(range(len(unique_label)), unique_label))

print("Create Mappings")
display(num_2_label_map)
display(label_2_num_map)

# print("Before Mappings")
# display(data[:, 1])
data[:,1] = np.vectorize(label_2_num_map.get)(data[:,1])
# print("After Mappings")
# display(data[:, 1])

Create Mappings


{0: 'payment',
 1: 'package',
 2: 'suspend',
 3: 'internet',
 4: 'phone_issues',
 5: 'service',
 6: 'nontruemove',
 7: 'balance',
 8: 'detail',
 9: 'bill',
 10: 'credit',
 11: 'promotion',
 12: 'mobile_setting',
 13: 'iservice',
 14: 'roaming',
 15: 'truemoney',
 16: 'information',
 17: 'lost_stolen',
 18: 'balance_minutes',
 19: 'idd',
 20: 'garbage',
 21: 'ringtone',
 22: 'rate',
 23: 'loyalty_card',
 24: 'contact',
 25: 'officer'}

{'payment': 0,
 'package': 1,
 'suspend': 2,
 'internet': 3,
 'phone_issues': 4,
 'service': 5,
 'nontruemove': 6,
 'balance': 7,
 'detail': 8,
 'bill': 9,
 'credit': 10,
 'promotion': 11,
 'mobile_setting': 12,
 'iservice': 13,
 'roaming': 14,
 'truemoney': 15,
 'information': 16,
 'lost_stolen': 17,
 'balance_minutes': 18,
 'idd': 19,
 'garbage': 20,
 'ringtone': 21,
 'rate': 22,
 'loyalty_card': 23,
 'contact': 24,
 'officer': 25}

In [70]:
def strip_str(string):
    return string.strip()
     
# print("Before")
# print(data)
data[:,0] = np.vectorize(strip_str)(data[:,0])
# print("After")
# print(data)

In [71]:
data

array([['<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counter Services เค้าเช็ต 3276.25 บาท เมื่อวานที่ผมเช็คที่ศูนย์บอกมียอด 3057.79 บาท',
        0],
       ['internet ยังความเร็วอยุ่เท่าไหร ครับ', 1],
       ['ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้ ค่ะ', 2],
       ...,
       ['ยอดเงินเหลือเท่าไหร่ค่ะ', 7],
       ['ยอดเงินในระบบ', 7],
       ['สอบถามโปรโมชั่นปัจจุบันที่ใช้อยู่ค่ะ', 1]], dtype=object)

In [72]:
vocab = defaultdict(lambda: len(vocab))
for i in tqdm(range(len(data))):
    tokens = attacut.tokenize(data[i][0])
    for token in tokens:
        vocab[token]
    
    data[i][0] = tokens
# 5447 unique tokens
print(len(vocab))

100%|██████████| 13389/13389 [04:25<00:00, 50.42it/s]

5447


In [ ]:
num_docs_with_w = defaultdict(int)
for i in tqdm(range(len(data))):
    unique_tokens = set(data[i][0])
    # should we remove any tokens?
    
    for token in unique_tokens:
        num_docs_with_w[vocab[token]] += 1
# sort token idx
num_docs_with_w = dict(sorted(num_docs_with_w.items(), key=lambda x: x[0]))

idf = np.log(len(data) / np.array(list(num_docs_with_w.values())))

100%|██████████| 13389/13389 [00:00<00:00, 126871.55it/s]


In [90]:
num_docs_with_w

{0: 428,
 1: 392,
 2: 420,
 3: 8674,
 4: 1991,
 5: 1976,
 6: 499,
 7: 1352,
 8: 2036,
 9: 1,
 10: 3,
 11: 1,
 12: 1,
 13: 125,
 14: 12,
 15: 428,
 16: 486,
 17: 149,
 18: 537,
 19: 51,
 20: 273,
 21: 1592,
 22: 1066,
 23: 1,
 24: 8,
 25: 149,
 26: 1451,
 27: 83,
 28: 49,
 29: 6,
 30: 469,
 31: 16,
 32: 3771,
 33: 11,
 34: 981,
 35: 1657,
 36: 1619,
 37: 2064,
 38: 885,
 39: 2961,
 40: 1031,
 41: 3290,
 42: 3428,
 43: 4395,
 44: 911,
 45: 870,
 46: 1043,
 47: 455,
 48: 2,
 49: 33,
 50: 33,
 51: 1205,
 52: 158,
 53: 181,
 54: 2375,
 55: 1206,
 56: 550,
 57: 548,
 58: 1570,
 59: 1397,
 60: 404,
 61: 474,
 62: 30,
 63: 30,
 64: 1,
 65: 598,
 66: 1172,
 67: 92,
 68: 1021,
 69: 1273,
 70: 1031,
 71: 109,
 72: 12,
 73: 1716,
 74: 15,
 75: 123,
 76: 913,
 77: 385,
 78: 400,
 79: 894,
 80: 270,
 81: 1,
 82: 902,
 83: 203,
 84: 4772,
 85: 340,
 86: 137,
 87: 493,
 88: 1201,
 89: 81,
 90: 417,
 91: 15,
 92: 284,
 93: 299,
 94: 26,
 95: 239,
 96: 1626,
 97: 2038,
 98: 16,
 99: 31,
 100: 3,
 101: 4

In [97]:
idf.shape

(5447,)

In [ ]:
def gen_tf_idf_for_sample():
    pass